In [6]:
import os
import numpy as np
import pandas as pd
import gc
import time
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import warnings
import numpy as np
import re
PATH = './'

total = pd.read_csv('./TEG 7-2.csv')
total.shape

(129, 62)

In [7]:
use = ['年龄', '主诊断（1oc2.EC 3.CC4.sarcoma）', '分化（1高分化2中分化3低分化4无分化）', '手术', '手术时长（min）',
       '术中输血1.有0无', '术中出血（ml）', '冠心病', '高血压', '高血脂', '血栓病史', '糖尿病', 'R', 'k', 'angle', 'MA',
       'CI', 'G', 'EPL', 'PT', 'INR', 'APTT', 'Fbg', 'D-D', 'T-T', 'R.1', 'k.1', 'angle.1', 'MA.1', 'CI.1', 'G.1',
       'EPL.1', 'PT.1', 'INR.1', 'APTT.1', 'FbG', 'D-D.1', 'T-T.1', '术前超声有无血栓(1有0无)']

df = total[use].rename(columns={'术前超声有无血栓(1有0无)':'l'})

df['R'].replace(' ', np.nan, inplace=True)
df['R'] = df['R'].apply(lambda x: float(x))
df.isnull().any()

年龄                            True
主诊断（1oc2.EC 3.CC4.sarcoma）    True
分化（1高分化2中分化3低分化4无分化）          True
手术                            True
手术时长（min）                     True
术中输血1.有0无                     True
术中出血（ml）                      True
冠心病                           True
高血压                           True
高血脂                           True
血栓病史                          True
糖尿病                           True
R                             True
k                             True
angle                         True
MA                            True
CI                            True
G                             True
EPL                           True
PT                            True
INR                           True
APTT                          True
Fbg                           True
D-D                           True
T-T                           True
R.1                           True
k.1                           True
angle.1                       True
MA.1                

In [3]:
cat_feats = ['主诊断（1oc2.EC 3.CC4.sarcoma）', 'l']
for col in cat_feats:
    df[col],_ = pd.factorize(df[col])
cat_feats = ['主诊断（1oc2.EC 3.CC4.sarcoma）', '分化（1高分化2中分化3低分化4无分化）', '手术', '手术时长（min）',
                     '术中输血1.有0无', '冠心病', '高血压', '高血脂', '血栓病史', '糖尿病']
num_feats = list(set(df.columns) - set(cat_feats + ['l']))
num_feats

['年龄',
 'k.1',
 'PT',
 'G.1',
 'MA.1',
 'R',
 'CI',
 'EPL.1',
 'INR',
 'APTT',
 'angle.1',
 'Fbg',
 '术中出血（ml）',
 'INR.1',
 'k',
 'angle',
 'D-D',
 'EPL',
 'G',
 'MA',
 'T-T',
 'CI.1',
 'R.1',
 'PT.1',
 'FbG',
 'D-D.1',
 'T-T.1',
 'APTT.1']

In [4]:
for c in set(df.columns) - set(['R','k', 'angle', 'MA',
       'CI', 'G', 'EPL', 'PT', 'INR', 'APTT', 'Fbg', 'D-D', 'T-T', 'R.1', 'k.1', 'angle.1', 'MA.1', 'CI.1', 'G.1',
       'EPL.1', 'PT.1', 'INR.1', 'APTT.1', 'FbG', 'D-D.1', 'T-T.1']):
    print(c, df[c].unique())

术中出血（ml） [  100.  1000.   800.  1400.    50.   500.    nan   200.   400.     0.
  1300.    80.   150.    10.    20.   300.   600.    30.  1200.]
高血压 [  0.   1.  nan]
糖尿病 [  0.   1.  nan]
年龄 [ 52.  65.  56.  67.  58.  55.  50.  66.  40.  59.  71.  36.  54.  47.  64.
  72.  53.  46.  73.  61.  28.  49.  51.  34.  62.  39.  33.  43.  16.  35.
  45.  24.  44.  29.  38.  42.  25.  63.  70.  75.  57.  32.  37.  48.  60.
  nan]
l [ 0 -1  1  2  3]
血栓病史 [  0.   1.  nan]
术中输血1.有0无 [  1.   0.  nan]
手术时长（min） [  85.  270.  315.  260.  180.   nan  120.  430.  150.  110.  165.  225.
   65.  140.   30.  190.  220.  130.  115.   90.   80.  200.  205.   50.
  100.  170.  145.  160.]
冠心病 [  0.   1.  nan]
分化（1高分化2中分化3低分化4无分化） [  1.   3.  nan   2.]
高血脂 [  0.   1.  nan]
主诊断（1oc2.EC 3.CC4.sarcoma） [ 0  1  2 -1  3  4]
手术 [  1.   2.  nan]


In [5]:
for c in num_feats:
    print(c, df[c].corr(df['l']))

年龄 0.141418711846
k.1 -0.0853386718984
PT -0.113649678012
G.1 -0.001004309893
MA.1 0.0142122300917
R -0.0694380092876
CI 0.10114510007
EPL.1 -0.211961637712
INR -0.0874545484223
APTT -0.0388274535086
angle.1 -0.0124110867874
Fbg 0.0198143115344
术中出血（ml） 0.218740966594
INR.1 -0.0815430541707
k -0.0103682796711
angle 0.0276292042056
D-D 0.169233190624
EPL -0.087675553785
G 0.104281640217
MA 0.0974870183066
T-T -0.459592102076
CI.1 -0.0202865028716
R.1 0.0727531831381
PT.1 -0.110456608706
FbG -0.00190553986483
D-D.1 -0.0518008187871
T-T.1 0.0131710220963
APTT.1 0.00769760647413


In [6]:
def ratio_name(col1, col2):
    return col1 + '_divide_' + col2
def ratio(df, col1, col2):
    return df[col1] / (df[col2] + 1e-3)
def times_name(col1, col2):
    return col1 + '_times_' + col2
def times(df, col1, col2):
    return df[col1] * df[col2]
def plus_name(col1, col2):
    return col1 + '_add_' + col2
def plus(df, col1, col2):
    return df[col1] + df[col2]
def minus_name(col1, col2):
    return col1 + '_minus_' + col2
def minus(df, col1, col2):
    return df[col1] - df[col2]


total_num_feats = num_feats.copy()
for x in num_feats:
    for y in num_feats:
        if ratio_name(y, x) not in df.columns and x != y:
            total_num_feats.append(ratio_name(x, y))
            df[ratio_name(x, y)] = ratio(df, x, y)
            
            total_num_feats.append(times_name(x, y))
            df[times_name(x, y)] = times(df, x, y)
            
            total_num_feats.append(plus_name(x, y))
            df[plus_name(x, y)] = plus(df, x, y)
            
            total_num_feats.append(minus_name(x, y))
            df[minus_name(x, y)] = minus(df, x, y)
total_num_feats

['年龄',
 'k.1',
 'PT',
 'G.1',
 'MA.1',
 'R',
 'CI',
 'EPL.1',
 'INR',
 'APTT',
 'angle.1',
 'Fbg',
 '术中出血（ml）',
 'INR.1',
 'k',
 'angle',
 'D-D',
 'EPL',
 'G',
 'MA',
 'T-T',
 'CI.1',
 'R.1',
 'PT.1',
 'FbG',
 'D-D.1',
 'T-T.1',
 'APTT.1',
 '年龄_divide_k.1',
 '年龄_times_k.1',
 '年龄_add_k.1',
 '年龄_minus_k.1',
 '年龄_divide_PT',
 '年龄_times_PT',
 '年龄_add_PT',
 '年龄_minus_PT',
 '年龄_divide_G.1',
 '年龄_times_G.1',
 '年龄_add_G.1',
 '年龄_minus_G.1',
 '年龄_divide_MA.1',
 '年龄_times_MA.1',
 '年龄_add_MA.1',
 '年龄_minus_MA.1',
 '年龄_divide_R',
 '年龄_times_R',
 '年龄_add_R',
 '年龄_minus_R',
 '年龄_divide_CI',
 '年龄_times_CI',
 '年龄_add_CI',
 '年龄_minus_CI',
 '年龄_divide_EPL.1',
 '年龄_times_EPL.1',
 '年龄_add_EPL.1',
 '年龄_minus_EPL.1',
 '年龄_divide_INR',
 '年龄_times_INR',
 '年龄_add_INR',
 '年龄_minus_INR',
 '年龄_divide_APTT',
 '年龄_times_APTT',
 '年龄_add_APTT',
 '年龄_minus_APTT',
 '年龄_divide_angle.1',
 '年龄_times_angle.1',
 '年龄_add_angle.1',
 '年龄_minus_angle.1',
 '年龄_divide_Fbg',
 '年龄_times_Fbg',
 '年龄_add_Fbg',
 '年龄_minus_Fbg',
 '年龄_di

In [9]:
p = []
for c in total_num_feats:
    tmp = df[c].corr(df['l'])
    p.append((tmp, c))
#     print(c, tmp)
p = pd.DataFrame(p, columns=['a', 'b'])
p.sort_values('a', ascending=False, inplace=True)
p

,a,b
1199,0.484485,k_minus_T-T
1147,0.462598,INR.1_minus_T-T
827,0.450754,INR_minus_T-T
1308,0.425455,D-D_divide_D-D.1
211,0.420816,k.1_minus_T-T
587,0.343809,R_minus_T-T
671,0.337311,CI_minus_T-T
1396,0.324521,MA_divide_T-T
1304,0.314749,D-D_divide_FbG
1108,0.312515,术中出血（ml）_divide_D-D.1


In [10]:
a = ['术中输血1.有0无', '冠心病', '高血压', '高血脂', '血栓病史', '糖尿病']
p = []
for c in a:
    tmp = df[c].corr(df['l'])
    p.append((tmp, c))
p = pd.DataFrame(p, columns=['a', 'b'])
p.sort_values('a', ascending=False, inplace=True)
p

,a,b
0,0.277002,术中输血1.有0无
4,0.128291,血栓病史
2,0.099776,高血压
5,-0.021331,糖尿病
1,-0.030818,冠心病
3,-0.094151,高血脂


In [ ]:
lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=cat_feats,
                          free_raw_data=False)
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.02,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.15,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 15.172643147364937}
cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

In [ ]:
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('gain'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('split'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x